In [14]:
def resolve_arg(program, argmode, base_offset, i, j):
    if argmode == 0:
        return program[i+j]
    elif argmode == 1:
        return i+j
    elif argmode == 2:
        return base_offset + program[i+j]

    
def intcode(program, inlist=[], i=0):
    outbuf = []
    base_offset = 0
    
    while True:
        if i >= len(program):
            print('end of program')
            return program
        opcode = program[i]
        
        op = opcode % 100
        argmodes = []
        argcode = opcode // 100
        while argcode > 0:
            argmodes.append(argcode % 10)
            argcode = argcode // 10
            
        # ADD A B -> C
        # --------------------------------------------------------
        if op == 1:
            if len(argmodes) < 1:
                idx1 = resolve_arg(program, 0, base_offset, i, 1)
            else:
                idx1 = resolve_arg(program, argmodes[0], base_offset, i, 1)
            
            if len(argmodes) < 2:
                idx2 = resolve_arg(program, 0, base_offset, i, 2)
            else:
                idx2 = resolve_arg(program, argmodes[1], base_offset, i, 2)
            
            if len(argmodes) < 3:
                idx3 = resolve_arg(program, 0, base_offset, i, 3)
            else:
                idx3 = resolve_arg(program, argmodes[2], base_offset, i, 3)

            bigdex = max([idx1, idx2, idx3])
            if len(program) <= bigdex:
                program += [0]*(bigdex - len(program) + 1)
                
            arg1 = program[idx1]
            arg2 = program[idx2]                
            
            program[idx3] = arg1 + arg2
            i += 4
        # MULTIPLY A B ->
        # --------------------------------------------------------
        elif op == 2:
            if len(argmodes) < 1:
                idx1 = resolve_arg(program, 0, base_offset, i, 1)
            else:
                idx1 = resolve_arg(program, argmodes[0], base_offset, i, 1)
            
            
            if len(argmodes) < 2:
                idx2 = resolve_arg(program, 0, base_offset, i, 2)
            else:
                idx2 = resolve_arg(program, argmodes[1], base_offset, i, 2)

            if len(argmodes) < 3:
                idx3 = resolve_arg(program, 0, base_offset, i, 3)
            else:
                idx3 = resolve_arg(program, argmodes[2], base_offset, i, 3)
                
            bigdex = max([idx1, idx2, idx3])
            if len(program) <= bigdex:
                program += [0]*(bigdex - len(program) + 1)
                
            arg1 = program[idx1]
            arg2 = program[idx2]
            
            program[idx3] = arg1 * arg2
            i += 4
        # INPUT -> A
        # --------------------------------------------------------
        elif op == 3:
            argmode = 0 if len(argmodes) < 1 else argmodes[0]
            idx = resolve_arg(program, argmode, base_offset, i, 1)
            
            if len(program) <= idx:
                program += [0]*(idx - len(program) + 1)

            if len(inlist) > 0:
                program[idx] = inlist[0]
                inlist = inlist[1:]
            else:
                instr = input('>>> ')
                inlist = [ord(e) for e in instr]
                inlist += [ord('\n')]
                program[idx] = inlist[0]
                inlist = inlist[1:]
#             print('input', program[idx])
            i += 2
        # OUTPUT A
        # --------------------------------------------------------
        elif op == 4:
            argmode = 0 if len(argmodes) < 1 else argmodes[0]
            idx = resolve_arg(program, argmode, base_offset, i, 1)
#             print('output', program[idx])
            outbuf.append(program[idx])
            if program[idx] == ord('\n'):
                print(''.join([chr(o) for o in outbuf[:-1]]))
                outbuf = []
            i += 2
        # JUMP TRUE A != 0 -> B
        # --------------------------------------------------------
        elif op == 5:                
            argmode = 0 if len(argmodes) < 1 else argmodes[0]
            idx1 = resolve_arg(program, argmode, base_offset, i, 1)
            
            argmode = 0 if len(argmodes) < 2 else argmodes[1]
            idx2 = resolve_arg(program, argmode, base_offset, i, 2)
            
            bigdex = max([idx1, idx2])
            if len(program) <= bigdex:
                program += [0]*(bigdex - len(program) + 1)
            
            arg1 = program[idx1]
            arg2 = program[idx2]    
            if not arg1 == 0:
                i = arg2
            else:
                i += 3
        # JUMP FALSE A == 0 -> B
        # --------------------------------------------------------
        elif op == 6:
            argmode = 0 if len(argmodes) < 1 else argmodes[0]
            idx = resolve_arg(program, argmode, base_offset, i, 1)
            arg1 = program[idx]
            
            argmode = 0 if len(argmodes) < 2 else argmodes[1]
            idx = resolve_arg(program, argmode, base_offset, i, 2)
            arg2 = program[idx]
            
            if len(program) <= arg2:
                program += [0]*(arg2-len(program)+1)
                
            if arg1 == 0:
                i = arg2
            else:
                i += 3
        # LESS THAN A < B 1 -> C
        # --------------------------------------------------------
        elif op == 7:
            argmode = 0 if len(argmodes) < 1 else argmodes[0]
            idx = resolve_arg(program, argmode, base_offset, i, 1)
            arg1 = program[idx]
            
            argmode = 0 if len(argmodes) < 2 else argmodes[1]
            idx = resolve_arg(program, argmode, base_offset, i, 2)
            arg2 = program[idx]
            
            argmode = 0 if len(argmodes) < 3 else argmodes[2]
            idx = resolve_arg(program, argmode, base_offset, i, 3)
                
            if len(program) <= idx:
                program += [0]*(idx-len(program)+1)
                
            if arg1 < arg2:
                program[idx] = 1
            else:
                program[idx] = 0
            i += 4
        # EQUALS A == B 1 -> C
        # --------------------------------------------------------
        elif op == 8:
            argmode = 0 if len(argmodes) < 1 else argmodes[0]
            idx = resolve_arg(program, argmode, base_offset, i, 1)
            arg1 = program[idx]
            
            argmode = 0 if len(argmodes) < 2 else argmodes[1]
            idx = resolve_arg(program, argmode, base_offset, i, 2)
            arg2 = program[idx]
            
            argmode = 0 if len(argmodes) < 3 else argmodes[2]
            idx = resolve_arg(program, argmode, base_offset, i, 3)
                
            if len(program) <= idx:
                program += [0]*(idx-len(program)+1)
                
            if arg1 == arg2:
                program[idx] = 1
            else:
                program[idx] = 0
            i += 4
        # SHIFT RELATIVE BASE
        # --------------------------------------------------------
        elif op == 9:
            argmode = 0 if len(argmodes) < 1 else argmodes[0]
            idx = resolve_arg(program, argmode, base_offset, i, 1)
            arg1 = program[idx]
            base_offset += arg1
            if len(program) <= base_offset:
                program += [0]*(base_offset-len(program)+1)
            i += 2
        # HALT
        # --------------------------------------------------------
        elif op == 99:
            return outbuf
        # INVALID
        # --------------------------------------------------------
        else:
            print(program[i], 'is not an instruction')
            return outbuf

In [20]:
with open('calca_25_input.txt', 'r') as infile:
    program = [[int(col) for col in row.split(',')] for row in infile if len(row) > 0][0]
    
intcode(program)




== Hull Breach ==
You got in through a hole in the floor here. To keep your ship from also freezing, the hole has been sealed.

Doors here lead:
- north
- east

Command?
>>> east



== Crew Quarters ==
The beds are all too small for you.

Doors here lead:
- west

Items here:
- antenna

Command?
>>> take antenna

You take the antenna.

Command?
>>> west



== Hull Breach ==
You got in through a hole in the floor here. To keep your ship from also freezing, the hole has been sealed.

Doors here lead:
- north
- east

Command?
>>> north



== Hallway ==
This area has been optimized for something; you're just not quite sure what.

Doors here lead:
- north
- south

Items here:
- weather machine

Command?
>>> take weather machine

You take the weather machine.

Command?
>>> north



== Storage ==
The boxes just contain more boxes.  Recursively.

Doors here lead:
- east
- south

Items here:
- klein bottle

Command?
>>> take klein bottle

You take the klein bottle.

Command?
>>> east



== St


You drop the antenna.

Command?
>>> east



== Pressure-Sensitive Floor ==
Analyzing...

Doors here lead:
- west

A loud, robotic voice says "Alert! Droids on this ship are heavier than the detected value!" and you are ejected back to the checkpoint.



== Security Checkpoint ==
In the next room, a pressure-sensitive floor will verify your identity.

Doors here lead:
- north
- east

Items here:
- shell
- weather machine
- antenna
- mug
- cake

Command?
>>> take shell

You take the shell.

Command?
>>> east



== Pressure-Sensitive Floor ==
Analyzing...

Doors here lead:
- west

A loud, robotic voice says "Alert! Droids on this ship are heavier than the detected value!" and you are ejected back to the checkpoint.



== Security Checkpoint ==
In the next room, a pressure-sensitive floor will verify your identity.

Doors here lead:
- north
- east

Items here:
- weather machine
- antenna
- mug
- cake

Command?
>>> take mug

You take the mug.

Command?
>>> east



== Pressure-Sensitive Flo

[]